!git clone https://github.com/ggerganov/llama.cpp

In [1]:
!pip install -r ./llama.cpp/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
  Obtaining dependency information for numpy~=1.26.4 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.2 MB/s eta 0:00:00
  Obtaining dependency information for transformers<5.0.0,>=4.40.1 from https://files.pythonhosted.org/packages/75/35/07c9879163b603f0e464b0f6e6e628a2340cfc7cdc5ca8e7d52d776710d4/transformers-4.44.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ---------------------------------------- 43.7/43.7 kB ? eta 0:00:00
  Obtaining dependency information for gguf>=0.1.0 from https://files

'C:\Users\Vamshi' is not recognized as an internal or external command,
operable program or batch file.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.8 requires protobuf<4,>=3.11, but you have protobuf 4.25.1 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
torchaudio 2.0.2 requires torch==2.0.1, but you have torch 2.2.2+cpu which is incompatible.
torchvision 0.16.0 requires torch==2.1.0, but you have torch 2.2.2+cpu which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r ./llama.cpp/requirements.txt

### download model from hub

In [4]:
from huggingface_hub import snapshot_download

In [4]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

In [5]:
methods = ['q4_k_s','q4_k_m']

In [6]:
base_model = "./original_model/"
quantized_path = "./quantized_model/"

In [7]:
from huggingface_hub import login
login(token="<your_token>")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [8]:
snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+'/FP16.gguf'

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1212: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

In [9]:
!mkdir ./quantized_model/

mkdir: cannot create directory ‘./quantized_model/’: File exists


### convert hf model to gguf format

In [17]:
!python llama.cpp/convert_hf_to_gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00003.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {4096, 32768}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {14336, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> F16, shape = {4096, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> F16, 

### quantize gguf models

In [5]:
import os

In [30]:
for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    print("source model: "+quantized_path+" \n destination model: /FP16.gguf "+qtype)
    os.system("!llama.cpp/examples/convert_legacy_llama.py "+quantized_path+"/FP16.gguf "+qtype+" "+m)

sh: !llama.cpp/examples/convert_legacy_llama.py: No such file or directory
sh: !llama.cpp/examples/convert_legacy_llama.py: No such file or directory


In [15]:
! ./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt

Log start
main: build = 2318 (4d4d2366)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: seed  = 1709401432
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla V100-SXM2-16GB, compute capability 7.0, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./quantized_model/Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = original_model
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 24
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_model_loader: - kv 

### push quantized models to huggingface repo

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [25]:
model_path = "quantized_model/FP16.gguf" # Your model's local path

In [22]:
repo_name = "Mistral-7B-Instruct-v0.3-FP16.gguf"  # Desired HF Hub repository name
repo_url = create_repo(repo_name, private=False)

HfHubHTTPError: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-66ab537b-6f5fcd4a68cbbfdd7d14c1f4;64fdeb1c-4fa1-4a8a-8a2b-3cd96f49cc7e)

You already created this model repo

In [26]:
api = HfApi()
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Mistral-7B-Instruct-v0.3-FP16.gguf",
    repo_id="vamshigvk/Mistral-7B-Instruct-v0.3-FP16.gguf",
    repo_type="model",
)

FP16.gguf:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vamshigvk/Mistral-7B-Instruct-v0.3-FP16.gguf/commit/05764e9089490cd71be7e43c51b462f45183ac44', commit_message='Upload Mistral-7B-Instruct-v0.3-FP16.gguf with huggingface_hub', commit_description='', oid='05764e9089490cd71be7e43c51b462f45183ac44', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
from huggingface_hub import metadata_update
metadata_update("vamshigvk/Mistral-7B-Instruct-v0.3-FP16.gguf", {"pipeline_tag": "text-generation"}, overwrite=True)

README.md:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


CommitInfo(commit_url='https://huggingface.co/vamshigvk/Mistral-7B-Instruct-v0.3-FP16.gguf/commit/c4d60d68a8e4fa6811ca9228bf599d5b8d5c5e2e', commit_message='Update metadata with huggingface_hub', commit_description='', oid='c4d60d68a8e4fa6811ca9228bf599d5b8d5c5e2e', pr_url=None, pr_revision=None, pr_num=None)